Version 1

In [12]:
import random
import pandas as pd
from faker import Faker

# Inisialisasi Faker
faker = Faker()

# Fungsi untuk membersihkan nama dan membuat email
def generate_email(name):
    first_name, last_name = name.split(" ")[:2]
    return f"{first_name.lower()}_{last_name.lower()}@example.com"

# Fungsi untuk Generate Data Dummy
def generate_customers(n=50):
    customers = []
    for i in range(1, n + 1):
        name = faker.name()
        customers.append({
            "id": i,
            "name": name,
            "email": generate_email(name),
            "phone": faker.msisdn(),
            "address": faker.address(),
            "created_at": faker.date_time_this_year().strftime("%Y-%m-%d %H:%M:%S"),
        })
    return pd.DataFrame(customers)

def generate_products(n=20):
    categories = ["Electronics", "Books", "Clothing", "Home", "Toys"]
    products = []
    for i in range(1, n + 1):
        products.append({
            "id": i,
            "name": faker.word().capitalize(),
            "category": random.choice(categories),
            "price": round(random.uniform(10, 1000), 2),
            "stock": random.randint(0, 100),
        })
    return pd.DataFrame(products)

def generate_order_items(products, n=2000):
    order_items = []
    product_ids = products["id"].tolist()  # Ambil ID dari tabel products
    order_id = 1  # Mulai dengan order_id pertama
    order_count = 0  # Untuk menghitung berapa banyak item dalam satu order
    k = 1
    
    for i in range(1, n + 1):
        # Tentukan jumlah item per order secara acak (1 sampai 10)
        items_in_order = random.randint(1, 10) 
        
        # Pastikan produk tidak terduplikasi di dalam satu order
        chosen_product_ids = random.sample(product_ids, items_in_order) 
        
        for product_id in chosen_product_ids:
            product_price = products.loc[products["id"] == product_id, "price"].values[0]
            quantity = random.randint(1, 5)
            order_items.append({
                "id": k,
                "order_id": order_id,  # Semua produk ini akan memiliki order_id yang sama
                "product_id": product_id,
                "quantity": quantity,
                "price": round(product_price * quantity, 2),
            })
            i += 1  # Increment ID untuk order item
            k += 1

        order_count += 1
        order_id += 1  # Untuk setiap set produk, order_id bertambah

    return pd.DataFrame(order_items)

def generate_orders(order_items, customers):
    orders = []
    customer_ids = customers["id"].tolist()  # Ambil ID dari tabel customers
    order_ids = order_items["order_id"].unique()  # Ambil ID unik dari order_items
    for order_id in order_ids:
        total_amount = order_items[order_items["order_id"] == order_id]["price"].sum()
        orders.append({
            "id": order_id,
            "customer_id": random.choice(customer_ids),  # Pilih ID dari customers
            "order_date": faker.date_time_this_year().strftime("%Y-%m-%d %H:%M:%S"),
            "total_amount": round(total_amount, 2),
        })
    return pd.DataFrame(orders)

def generate_shipments(orders):
    statuses = ["Shipped", "Delivered", "Pending"]
    shipments = []
    order_ids = orders["id"].tolist()  # Ambil ID dari tabel orders
    for i, order_id in enumerate(order_ids, 1):
        shipments.append({
            "id": i,
            "order_id": order_id,  # Pilih ID dari orders
            "shipment_date": faker.date_time_this_year().strftime("%Y-%m-%d %H:%M:%S"),
            "status": random.choice(statuses),
            "tracking_number": faker.ean(length=13),
        })
    return pd.DataFrame(shipments)

# Simpan Data ke CSV
if __name__ == "__main__":
    # Generate Data
    customers = generate_customers()
    products = pd.read_csv("products.csv")
    order_items = generate_order_items(products)
    orders = generate_orders(order_items, customers)
    shipments = generate_shipments(orders)

    # Simpan ke CSV
    customers.to_csv("customers.csv", index=False)
    products.to_csv("products.csv", index=False)
    orders.to_csv("orders.csv", index=False)
    order_items.to_csv("order_items.csv", index=False)
    shipments.to_csv("shipments.csv", index=False)

    print("Data dummy berhasil disimpan ke file CSV!")


Data dummy berhasil disimpan ke file CSV!


Buat manual product nya

In [3]:
import json
import pandas as pd

# Membaca file JSON
with open('data.json', 'r') as file:
    data = json.load(file)

# Mengonversi data JSON ke DataFrame
df = pd.DataFrame(data)

# Menyimpan ke file CSV
df.to_csv('products.csv', index=False)

print("Data berhasil disimpan ke 'products.csv'!")


Data berhasil disimpan ke 'products.csv'!


Cek orders dan order_items

In [16]:
import pandas as pd

# Membaca file CSV
orders = pd.read_csv("orders.csv")
order_items = pd.read_csv("order_items.csv")

# Menghitung total harga untuk setiap order di order_items
order_items_total = order_items.groupby('order_id')['price'].sum().reset_index()
order_items_total.columns = ['order_id', 'calculated_total']

# Gabungkan dengan tabel orders berdasarkan order_id
orders_with_calculated_total = pd.merge(
    orders, 
    order_items_total, 
    left_on='id',  # Kolom di orders
    right_on='order_id',  # Kolom di order_items_total
    how='left'
)


# Bandingkan total_amount dengan calculated_total
orders_with_calculated_total['is_total_matching'] = orders_with_calculated_total['total_amount'] == orders_with_calculated_total['calculated_total']

# Tampilkan baris yang tidak cocok
mismatched_orders = orders_with_calculated_total[orders_with_calculated_total['is_total_matching'] == False]

# Jika ada ketidaksesuaian, tampilkan
if not mismatched_orders.empty:
    print("Ada ketidaksesuaian total pada order berikut:")
    print(mismatched_orders)
else:
    print("Semua total_amount dan calculated_total cocok.")


Ada ketidaksesuaian total pada order berikut:
        id  customer_id           order_date  total_amount  order_id  \
0        1            4  2025-01-08 00:10:23        428.93         1   
7        8           34  2025-01-06 16:55:36        409.95         8   
10      11            2  2025-01-06 13:08:28       1875.72        11   
13      14            1  2025-01-02 15:17:34       1304.84        14   
16      17           27  2025-01-04 12:20:21        989.84        17   
...    ...          ...                  ...           ...       ...   
1980  1981           12  2025-01-11 15:57:14        859.89      1981   
1982  1983           28  2025-01-04 00:34:28        869.91      1983   
1984  1985           45  2025-01-07 01:06:33       1982.78      1985   
1988  1989           31  2025-01-11 20:48:02        509.94      1989   
1998  1999           27  2025-01-03 06:25:45       1909.74      1999   

      calculated_total  is_total_matching  
0               428.93              False  
7

Cek SUm

In [17]:
# Hitung sum dari total_amount di tabel orders
sum_orders_total_amount = orders['total_amount'].sum()

# Hitung sum dari price di tabel order_items
sum_order_items_price = order_items['price'].sum()

# Tampilkan hasilnya
print(f"Total sum total_amount di orders: {sum_orders_total_amount}")
print(f"Total sum price di order_items: {sum_order_items_price}")


Total sum total_amount di orders: 1707934.02
Total sum price di order_items: 1707934.02


Final Version

In [ ]:
import random
import pandas as pd
from faker import Faker
from datetime import timedelta, datetime, date

# Inisialisasi Faker
faker = Faker()

# Fungsi untuk membersihkan nama dan membuat email
def generate_email(name):
    first_name, last_name = name.split(" ")[:2]
    return f"{first_name.lower()}_{last_name.lower()}@example.com"

# Fungsi untuk Generate Data Dummy
def generate_customers(n=150):
    customers = []
    for i in range(1, n + 1):
        name = faker.name()
        customers.append({
            "id": i,
            "name": name,
            "email": generate_email(name),
            "phone": faker.msisdn(),
            "address": faker.address(),
            # Parse the start and end date strings to datetime objects
            "created_at": faker.date_between(start_date=datetime(2020, 1, 1), end_date=datetime(2024, 12, 31)).strftime("%Y-%m-%d %H:%M:%S"),
        })
    return pd.DataFrame(customers)

def generate_products(n=20):
    categories = ["Electronics", "Books", "Clothing", "Home", "Toys"]
    products = []
    for i in range(1, n + 1):
        products.append({
            "id": i,
            "name": faker.word().capitalize(),
            "category": random.choice(categories),
            "price": round(random.uniform(10, 1000), 2),
            "stock": random.randint(0, 100),
        })
    return pd.DataFrame(products)

def generate_order_items(products, n=10000):
    order_items = []
    product_ids = products["id"].tolist()  # Ambil ID dari tabel products
    order_id = 1  # Mulai dengan order_id pertama
    k = 1
    
    for i in range(1, n + 1):
        # Tentukan jumlah item per order secara acak (1 sampai 10)
        items_in_order = random.randint(1, 10) 
        
        # Pastikan produk tidak terduplikasi di dalam satu order
        chosen_product_ids = random.sample(product_ids, items_in_order) 
        
        for product_id in chosen_product_ids:
            product_price = products.loc[products["id"] == product_id, "price"].values[0]
            quantity = random.randint(1, 5)
            order_items.append({
                "id": k,
                "order_id": order_id,  # Semua produk ini akan memiliki order_id yang sama
                "product_id": product_id,
                "quantity": quantity,
                "price": round(product_price * quantity, 2),
            })
            i += 1  # Increment ID untuk order item
            k += 1

        order_id += 1  # Untuk setiap set produk, order_id bertambah

    return pd.DataFrame(order_items)

def generate_orders(order_items, customers):
    orders = []
    customer_ids = customers["id"].tolist()  # Ambil ID dari tabel customers
    order_ids = order_items["order_id"].unique()  # Ambil ID unik dari order_items
    for order_id in order_ids:
        total_amount = order_items[order_items["order_id"] == order_id]["price"].sum()
        
        # Menghasilkan order_date antara 1 Januari 2024 dan 31 Desember 2024
        order_date = faker.date_between(start_date=date(2024, 1, 1), end_date=date(2024, 12, 31)).strftime("%Y-%m-%d %H:%M:%S")
        
        orders.append({
            "id": order_id,
            "customer_id": random.choice(customer_ids),  # Pilih ID dari customers
            "order_date": order_date,
            "total_amount": round(total_amount, 2),
        })
    return pd.DataFrame(orders)

def generate_shipments(orders):
    statuses = ["Shipped", "Delivered", "Pending"]
    shipments = []
    order_ids = orders["id"].tolist()  # Ambil ID dari tabel orders
    check_date = datetime(2024, 12, 31)  # Pengecekan akhir tahun

    for i, order_id in enumerate(order_ids, 1):
        order_date = orders[orders["id"] == order_id]["order_date"].values[0]
        order_date = datetime.strptime(order_date, "%Y-%m-%d %H:%M:%S")
        
        # Tentukan status: Jika order_date lebih dari 15 hari dari check_date, harus Delivered
        if (check_date - order_date).days > 15:
            status = "Delivered"
        else:
            # 80% Delivered, 15% Shipped, 5% Pending
            status = random.choices(statuses, weights=[80, 15, 5], k=1)[0]
        
        # Tentukan shipment_date dan delivery_date jika status Delivered
        shipment_date = None
        delivery_date = None
        if status != "Pending":
            # Shipment terjadi antara 1-7 hari setelah order date, lebih banyak di 1-5 hari
            shipment_date_range = random.choices(range(1, 8), weights=[5, 5, 5, 5, 5, 2, 1], k=1)[0]
            shipment_date = order_date + timedelta(days=shipment_date_range)

            if status == "Delivered":
                # Delivery terjadi antara 1-15 hari setelah shipment date, lebih banyak di 1-7 hari
                delivery_date_range = random.choices(range(1, 16), weights=[5, 5, 5, 5, 5, 3, 3, 3, 2, 2, 1, 1, 1, 1, 1], k=1)[0]
                delivery_date = shipment_date + timedelta(days=delivery_date_range)

        shipments.append({
            "id": i,
            "order_id": order_id,
            "shipment_date": shipment_date.strftime("%Y-%m-%d %H:%M:%S") if shipment_date else None,
            "status": status,
            "tracking_number": faker.ean(length=13) if status != "Pending" else None,
            "delivery_date": delivery_date.strftime("%Y-%m-%d") if delivery_date else None,
        })

    return pd.DataFrame(shipments)


# Simpan Data ke CSV
if __name__ == "__main__":
    # Generate Data
    customers = generate_customers()
    products = pd.read_csv("products.csv")
    order_items = generate_order_items(products)
    orders = generate_orders(order_items, customers)
    shipments = generate_shipments(orders)

    # Simpan ke CSV
    customers.to_csv("./data/customers.csv", index=False)
    products.to_csv("./data/products.csv", index=False)
    orders.to_csv("./data/orders.csv", index=False)
    order_items.to_csv("./data/order_items.csv", index=False)
    shipments.to_csv("./data/shipments.csv", index=False)

    print("Data dummy berhasil disimpan ke file CSV!")


Data dummy berhasil disimpan ke file CSV!
